In [67]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import ast
import pandas as pd

# Grab the 'menu' item from the American Kennel Club (AKC) website
dog_breed_main = requests.get('https://www.akc.org/dog-breeds/') 
soup = BeautifulSoup(dog_breed_main.content, 'html.parser')
menu = soup.find('div', {'class':'custom-select'})

In [68]:
# Then re-parse menu to get list of links to each breed
menu_soup = BeautifulSoup(str(menu), 'html.parser')
breed_results = menu_soup.find_all('option')[1:]
breed_links = [result['value'] for result in breed_results]

In [92]:
# Scrape each link for information

prefix = 'googletag.pubads().setTargeting'
dog_char_list = []

for link in tqdm(breed_links):
    breed_html = requests.get(link)
    bs = BeautifulSoup(breed_html.content, 'html.parser')
    js_ads = [script for script in bs.find_all('script') if "googletag.pubads()" in str(script)][0]
    pub_ads = [ad.strip()[len(prefix):-1] for ad in str(js_ads).split('\n') if 'googletag.pubads().setTargeting' in ad]
    dog_chars = [ast.literal_eval(char) for char in pub_ads] 
    dog_char_list.append(dog_chars)

100%|██████████| 281/281 [01:46<00:00,  2.64it/s]


In [99]:
dog_char_df = pd.DataFrame(columns=['Breed', 'Size', 'Group'])
dog_char_df_array = []
bad_links = ['spinone-italiano']

for dog_char in dog_char_list:
    dog_char_dict = {x:y for x,y in dog_char}  

    '''def parse_characteristics(chars):
    valid_coat_types = ['hairless', 'medium', 'smooth', 'short', 'long', 'medium']
    valid_bark_levels = ['when-necessary', 'medium', 'likes-to-be-vocal', 'infrequent', 'frequent']
    valid_trainabilities = ['may-be-stubborn', 'eager-to-please', 'easy-training', 'agreeable', 'independent']
    coat_type = [char for char in chars if char.startswith('coat-type') and ''.split('')]
    bark_lvl = [char for char in chars if char.startswith('barking-level')]
    temperament = [char for char in chars if char.startswith('temperament')]
    trainability = [char for char in chars if char.starswith('trainability')]
    '''
    print(dog_char_dict['breed'])
    
    if dog_char_dict['breed'] not in bad_links:
        df_row = [str.capitalize(dog_char_dict['breed']), dog_char_dict['size'], dog_char_dict['group']]
        dog_char_df_array.append(df_row)

print(dog_char_df_array)

affenpinscher
afghan-hound
airedale-terrier
akita
alaskan-klee-kai
alaskan-malamute
american-bulldog
american-english-coonhound
american-eskimo-dog
american-foxhound
american-hairless-terrier
american-leopard-hound
american-staffordshire-terrier
american-water-spaniel
anatolian-shepherd-dog
appenzeller-sennenhund
australian-cattle-dog
australian-kelpie
australian-shepherd
australian-stump-tail-cattle-dog
australian-terrier
azawakh
barbado-da-terceira
barbet
basenji
basset-fauve-de-bretagne
basset-hound
bavarian-mountain-scent-hound
beagle
bearded-collie
beauceron
bedlington-terrier
belgian-laekenois
belgian-malinois
belgian-sheepdog
belgian-tervuren
bergamasco-sheepdog
berger-picard
bernese-mountain-dog
bichon-frise
biewer-terrier
black-and-tan-coonhound
black-russian-terrier
bloodhound
bluetick-coonhound
boerboel
bohemian-shepherd
bolognese
border-collie
border-terrier
borzoi
boston-terrier
bouvier-des-flandres
boxer
boykin-spaniel
bracco-italiano
braque-du-bourbonnais
braque-francais

In [100]:
df = pd.DataFrame(data=dog_char_df_array, columns=["Breed", "Size", "Group"])

In [102]:
df.to_csv("")

                 Breed    Size                     Group
0        Affenpinscher  xsmall                       toy
1         Afghan-hound   large                     hound
2     Airedale-terrier  medium                   terrier
3                Akita  xlarge                   working
4     Alaskan-klee-kai  medium  foundation-stock-service
..                 ...     ...                       ...
275  Wirehaired-vizsla  medium                  sporting
276     Working-kelpie  medium  foundation-stock-service
277     Xoloitzcuintli  medium              non-sporting
278     Yakutian-laika  medium  foundation-stock-service
279  Yorkshire-terrier  xsmall                       toy

[280 rows x 3 columns]
